## Query structure mirrors pif structure
* You can query subsystems, processing steps, properties, conditions of properties etc. by creating a query that matches the object hierarchy to the section you want to query


In [1]:
from citrination_client import CitrinationClient, PifQuery, SystemQuery, Filter, ChemicalFieldOperation, ValueQuery, PropertyQuery, FieldOperation, ChemicalFilter
from pypif import pif
from os import environ

## Flattening the PIF structure

extract_as creates a flattened dictionary structure mapping user supplied keys to objects in the PIF that match within the query

extract_all is an option for extract_as that pulls a list of all objects at the level in the hierarchy that match the query

In [2]:
client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')

dataset_id = 150713

critical_stress = PropertyQuery(name=FieldOperation(filter=Filter(equal="Critical Stress")),
                                units=FieldOperation(filter=Filter(equal="MPa")),
                                value=FieldOperation(extract_as="Critical stress"))

search_result = client.search(PifQuery(include_datasets=[dataset_id], system=SystemQuery(properties=[critical_stress])))

print "{} hits".format(search_result.total_num_hits)
print "Extracted fields:"
for i in range(search_result.total_num_hits):
    print pif.dumps(search_result.hits[i].extracted, indent=4)

1 hits
Extracted fields:
{
    "Critical stress": "677.7138222"
}


In [131]:
resistivity = PropertyQuery(name=FieldOperation(filter=Filter(equal="Resistivity")),
                            units=FieldOperation(filter=Filter(equal="Ohm*cm")),
                            value=FieldOperation(extract_as="Resistivity", extract_all=True),
                            conditions=ValueQuery(name=FieldOperation(filter=Filter(equal="Temperature")),
                                                  units=FieldOperation(filter=Filter(equal="K")),
                                                  value=FieldOperation(extract_as="Temperature", extract_all=True))
                            )

search_result = client.search(PifQuery(include_datasets=[dataset_id], 
                                       system=SystemQuery(chemical_formula=ChemicalFieldOperation(extract_as="formula"),
                                       properties=[resistivity])))

print "{} hits".format(search_result.total_num_hits)
print "Extracted fields:"

for i in range(search_result.total_num_hits):
    print pif.dumps(search_result.hits[i].extracted, indent=4)

1 hits
Extracted fields:
{
    "formula": "Li0.0024Ni0.9976O", 
    "Resistivity": [
        "28.8677", 
        "0.2629", 
        "0.0466"
    ], 
    "Temperature": [
        "400", 
        "700", 
        "1000"
    ]
}


## Chemical formula search
Citrine has developed specialized search functionality specifically for chemical formulas. The analyser parses the search string and recognizes chemical entities such as elements and stoichiometries to find chemically relevant results.


In [3]:
mp_dataset_id = 150675

query = PifQuery(include_datasets=[mp_dataset_id],
                 system=SystemQuery(chemical_formula=ChemicalFieldOperation(extract_as="formula")),
                 random_results = True,
                 size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)
    

query = PifQuery(include_datasets=[150675],
             system=SystemQuery(chemical_formula=ChemicalFieldOperation(
                filter=ChemicalFilter(equal='Ga'),
                extract_as="formula"
             )),
             random_results = True,
             size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)


query = PifQuery(include_datasets=[150675],
                 system=SystemQuery(chemical_formula=ChemicalFieldOperation(
                    filter=ChemicalFilter(equal='?x?y?z'),
                    extract_as="formula"
                 )),
                 random_results = True,
                 size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)

    
    
query = PifQuery(include_datasets=[150675],
                 system=SystemQuery(chemical_formula=ChemicalFieldOperation(
                    filter=ChemicalFilter(equal='?xOy'),
                    extract_as="formula"
                 )),
                 random_results = True,
                 size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)
    
    
query = PifQuery(include_datasets=[150675],
                 system=SystemQuery(chemical_formula=ChemicalFieldOperation(
                    filter=ChemicalFilter(equal='?1O1'),
                    extract_as="formula"
                 )),
                 random_results = True,
                 size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)

52265 hits
{"formula": "CaZn"}
{"formula": "Ba3Pb5"}
{"formula": "Ba2PbO4"}
{"formula": "RbC"}
{"formula": "GaMo4CS8"}
46 hits
{"formula": "Ga"}
{"formula": "GaS"}
{"formula": "Ga(MoS2)4"}
{"formula": "Ga(MoS2)4"}
{"formula": "GaS"}
22860 hits
{"formula": "SrCuSb"}
{"formula": "Li7NiN4"}
{"formula": "CaV3O7"}
{"formula": "HoNbO4"}
{"formula": "Pr3AlC"}
2856 hits
{"formula": "Ho2O3"}
{"formula": "ZrO2"}
{"formula": "Bi2O3"}
{"formula": "CoO2"}
{"formula": "SiO2"}
438 hits
{"formula": "NiO"}
{"formula": "SnO"}
{"formula": "MgO"}
{"formula": "BaO"}
{"formula": "MgO"}


## Logical operations

SHOULD, MUST, OPTIONAL, MUST_NOT

In [5]:
query = PifQuery(include_datasets=[150675],
         system=SystemQuery(chemical_formula=[ChemicalFieldOperation(
            filter=ChemicalFilter(equal='?xOy'),
            extract_as="formula"
         ),
         ChemicalFieldOperation(
            filter=ChemicalFilter(equal='?1O1', logic="MUST_NOT"),
            extract_as="formula"
         )]),
         random_results = True,
         size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted)

query = PifQuery(include_datasets=[150675],
         system=SystemQuery(chemical_formula=ChemicalFieldOperation(extract_as="formula"),
         properties=[PropertyQuery(name=FieldOperation(filter=Filter(equal="Enthalpy of Formation")),
                                   value=FieldOperation(extract_as="H_f"),
                                   logic="SHOULD"
                                  ),
                     PropertyQuery(name=FieldOperation(filter=Filter(equal="Band Gap")),
                                   value=FieldOperation(extract_as="Band gap",
                                                       filter=Filter(min=1E-5)),
                                   logic="MUST"
                                  ),
                     PropertyQuery(name=FieldOperation(filter=Filter(equal="Poisson Ratio")),
                                   value=FieldOperation(extract_as="Poisson Ratio"),
                                   logic="SHOULD"
                                  )
                    ]
         ),
         random_results = True,
         size=5)

search_result = client.search(query)
print "{} hits".format(search_result.total_num_hits)
for i in range(5):
    print pif.dumps(search_result.hits[i].extracted, indent=4)

2418 hits
{"formula": "Pr7O12"}
{"formula": "SiO2"}
{"formula": "Nd2O3"}
{"formula": "SiO2"}
{"formula": "H2O"}
52265 hits
{
    "H_f": "-0.1983459014950011", 
    "formula": "Gd(C2N3)3", 
    "Band gap": ""
}
{
    "H_f": "-1.0249150151302089", 
    "formula": "RbUAgSe3", 
    "Band gap": ""
}
{
    "H_f": "-0.587220543333333", 
    "formula": "Ga2Pt", 
    "Poisson Ratio": "0.40899863035664524", 
    "Band gap": "0.0"
}
{
    "H_f": "-0.4788030565151512", 
    "formula": "La2Ni5B4", 
    "Band gap": ""
}
{
    "H_f": "-0.16859844000000002", 
    "formula": "HgTe", 
    "Poisson Ratio": "0.35258469053026087", 
    "Band gap": "0.0"
}
